In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('NonfictionCirc.csv')
print(df.info())
df.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   BrowseAuthor                1798 non-null   object 
 1   BrowseTitle                 1974 non-null   object 
 2   PublicationYear             1972 non-null   float64
 3   CallNumber                  1974 non-null   object 
 4   TotalItemLifetimeCircCount  1974 non-null   int64  
 5   TotalItemYTDCircCount       1974 non-null   int64  
 6   Unnamed: 6                  0 non-null      float64
 7   Call No. Color Code         11 non-null     object 
dtypes: float64(2), int64(2), object(4)
memory usage: 123.5+ KB
None


,BrowseAuthor,BrowseTitle,PublicationYear,CallNumber,TotalItemLifetimeCircCount,TotalItemYTDCircCount,Unnamed: 6,Call No. Color Code
0,"Obama, Michelle, 1964- author.",Becoming,2018.0,"BIOG OBAMA,M OBA",160,5,NaN,BIOG
1,"Westover, Tara, author.",Educated : a memoir,2018.0,"BIOG WESTOVER,T WES",103,7,NaN,0
2,"Wilkerson, Isabel, author.",Caste : the origins of our discontents,2020.0,305.5 WIL,76,3,NaN,100
3,"Clear, James, author.","Atomic habits : tiny changes, remarkable resul...",2018.0,155.24 CLE,66,12,NaN,200
4,"Carnegie, Dale, 1888-1955.",How to win friends and influence people,2009.0,158.1 CAR,52,2,NaN,300
5,"Tolle, Eckhart, 1948-",The power of now : a guide to spiritual enligh...,2004.0,291.44 TOL,48,5,NaN,400
6,"Didion, Joan, author.",Slouching towards Bethlehem,2008.0,814.54 DID,48,4,NaN,500
7,"Graham, Benjamin, 1894-1976.",The intelligent investor : the classic text on...,2005.0,332.67 GRA,48,3,NaN,600
8,NhÃ¢Ì,The miracle of mindfulness : an introduction t...,2016.0,294.3 NHA,48,1,NaN,700
9,NaN,Good housekeeping Instant Pot cookbook : 60 de...,2018.0,641.587 GOO,47,5,NaN,800
